<a href="https://colab.research.google.com/github/ChicagoPark/chicago_work/blob/main/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Necessary Modules
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import os
from keras.preprocessing.image import ImageDataGenerator # Data Augmentation(Image transformations) 을 하기 위해 필요한 모듈

import scipy
import keras
from tensorflow.keras.utils import plot_model
! pip install keras_tuner
import keras_tuner as kt
import shutil
! pip install tensorflow-gpu


In [2]:
# Colab 상의 훈련용 데이터 디렉터리 정리
!mkdir training_data
%cd training_data/
!mkdir '0. stone'
!mkdir '1. pothole'
!mkdir '2. filled'
%cd ..

/content/training_data
/content


In [3]:
'''
# data path
# 인터넷에서 주워온것임 ! gdown --id 1PGhXUnaJDpcjgoLEhNn9gRvGZGRCzWUt
'''

# 빅데이터 AI 수업에서 공유된 data.zip 
#! gdown --id 1X1J2RwBu9KhRpw8GqS-s1ZeI6YF0VZTO --out 'data.zip'

# data.zip 학교제공 : 1X1J2RwBu9KhRpw8GqS-s1ZeI6YF0VZTO / 내 드라이브 : 1FB_Ssfk9A06wUFK5TlKN5tjKHaGOPsAA

# 빅데이터 AI 수업에서 공유된 실차 데이터.zip
#! gdown --id 12PwxkFYqIBiI_7Upxfyxg9Aq6MbmO2Wm --out '실차데이터.zip'

# 실차데이터 학교제공 : 1byD-TlXFXVJysi7q3K8y-W_96qbbfF5L / 내 드라이브 : 12PwxkFYqIBiI_7Upxfyxg9Aq6MbmO2Wm

# json parsing 에서 나누었던 데이터.zip
#! gdown --id 1Dr8z1zXNpTZVqcKd1cYn0d71gi5itA3W --out 'divided_data.zip'

# 하이퍼파라미터 설정을 위한 tuner_data.zip 파일
! gdown --id 1J7d62lIOboSWb2Tt79bqHuFJJRa-k9uM --out 'tuner_data.zip'

# 직접 데이터 추출 및 작업을 진행한 구글드라이브 상 데이터
# ! gdown --id <추가예정> --out '<설정하고자하는폴더명>.zip'

'''
data.zip 파일은 큰 파일이니 아래 절차로 zip 해제할 것
! zip -FFv data.zip --out data_.zip
! unzip data_.zip -d data
'''

!unzip 'tuner_data.zip' -d 'tuner_data'

'''
!unzip '실차데이터.zip' -d '실차데이터'
!unzip 'divided_data.zip' -d 'divided_data'

! zip -FFv data.zip --out data_.zip
! unzip data_.zip -d data
'''

Downloading...
From: https://drive.google.com/uc?id=1J7d62lIOboSWb2Tt79bqHuFJJRa-k9uM
To: /content/tuner_data.zip
100% 4.52G/4.52G [00:54<00:00, 82.8MB/s]
Archive:  tuner_data.zip
   creating: tuner_data/0. stone/
  inflating: tuner_data/0. stone/100490.png  
  inflating: tuner_data/0. stone/100880.png  
  inflating: tuner_data/0. stone/101490.png  
  inflating: tuner_data/0. stone/102100.png  
  inflating: tuner_data/0. stone/102450.png  
  inflating: tuner_data/0. stone/102810.png  
  inflating: tuner_data/0. stone/103340.png  
  inflating: tuner_data/0. stone/103800.png  
  inflating: tuner_data/0. stone/104080.png  
  inflating: tuner_data/0. stone/104700.png  
  inflating: tuner_data/0. stone/105440.png  
  inflating: tuner_data/0. stone/105850.png  
  inflating: tuner_data/0. stone/10620.png  
  inflating: tuner_data/0. stone/107790.png  
  inflating: tuner_data/0. stone/107990.png  
  inflating: tuner_data/0. stone/108180.png  
  inflating: tuner_data/0. stone/108490.png  
  inf

"\n!unzip '실차데이터.zip' -d '실차데이터'\n!unzip 'divided_data.zip' -d 'divided_data'\n\n! zip -FFv data.zip --out data_.zip\n! unzip data_.zip -d data\n"

In [ ]:
! gdown --id 1FB_Ssfk9A06wUFK5TlKN5tjKHaGOPsAA --out 'data.zip'
! zip -FFv data.zip --out data_.zip
! unzip data_.zip -d data

In [4]:
# training_data 폴더로 zip 해제한 모든 파일을 정리한다.
!mv /content/실차데이터/'0. no_pothole'/* /content/training_data/'0. stone'
!mv /content/실차데이터/'1. pothole'/* /content/training_data/'1. pothole'
!mv /content/실차데이터/'2. filled'/* /content/training_data/'2. filled'

!mv /content/divided_data/'0. stone/Images'/* /content/training_data/'0. stone'
!mv /content/divided_data/'1. pothole/Images'/* /content/training_data/'1. pothole'
!mv /content/divided_data/'2. filled/Images'/* /content/training_data/'2. filled'

!mv /content/data/data/'0. stone'/* /content/training_data/'0. stone'
!mv /content/data/data/'1. pothole'/* /content/training_data/'1. pothole'
!mv /content/data/data/'2. filled'/* /content/training_data/'2. filled'

!mv /content/tuner_data/'0. stone'/* /content/training_data/'0. stone'
!mv /content/tuner_data/'1. pothole'/* /content/training_data/'1. pothole'
!mv /content/tuner_data/'2. filled'/* /content/training_data/'2. filled'

mv: cannot stat '/content/실차데이터/0. no_pothole/*': No such file or directory
mv: cannot stat '/content/실차데이터/1. pothole/*': No such file or directory
mv: cannot stat '/content/실차데이터/2. filled/*': No such file or directory
mv: cannot stat '/content/divided_data/0. stone/Images/*': No such file or directory
mv: cannot stat '/content/divided_data/1. pothole/Images/*': No such file or directory
mv: cannot stat '/content/divided_data/2. filled/Images/*': No such file or directory
mv: cannot stat '/content/data/data/0. stone/*': No such file or directory
mv: cannot stat '/content/data/data/1. pothole/*': No such file or directory
mv: cannot stat '/content/data/data/2. filled/*': No such file or directory


In [16]:
# 1. 기본 뿌리 디렉터리와 검증데이터 경로를 가져온다
base_path = os.path.abspath('/content/training_data')
print(base_path)

# 2. 데이터 경로를 지정을 해준다.
no_pothole_dir = os.path.join(base_path, '0. stone')
pothole_dir = os.path.join(base_path, '1. pothole')
filled_dir = os.path.join(base_path, '2. filled')

# 3. 데이터 셋의 개수를 카운팅한다.
num_no_pothole = len(os.listdir(no_pothole_dir))
num_pothole_dir = len(os.listdir(pothole_dir))
num_filled_dir = len(os.listdir(filled_dir))

total_data = num_no_pothole + num_pothole_dir + num_filled_dir

print(f'정상도로 : {num_no_pothole} 개 , 팟홀도로 : {num_pothole_dir}, 보수도로 : {num_filled_dir}')

print(f'Total Data 개수 : {total_data}')

/content/training_data
정상도로 : 1000 개 , 팟홀도로 : 1000, 보수도로 : 1001
Total Data 개수 : 3001


In [17]:
# Train set 과 Validation set 을 나누며, Data Augmentation 작업 또한 추가시킨다.
train_image_generator = ImageDataGenerator(rescale=1./255,
      #rotation_range=8,
      width_shift_range=0.1,
      height_shift_range=0.1,
      #shear_range=0.1,
      #zoom_range=0.1,
      horizontal_flip=True,
      fill_mode='nearest',
      validation_split=0.1)
validation_image_generator = ImageDataGenerator(rescale=1./255,
                                                validation_split=0.15)
batch_size = 32
epochs = 15
IMG_HEIGHT = 320
IMG_WIDTH = 320

In [18]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=base_path,
                                                           seed = 42,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           subset='training',
                                                           class_mode='sparse')

val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=base_path,
                                                           seed = 42,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           subset='validation',
                                                           class_mode='sparse')
train_num = train_data_gen.samples # training set 이 몇개인지
val_num = val_data_gen.samples   # validation set 이 몇개인지

Found 2700 images belonging to 3 classes.
Found 450 images belonging to 3 classes.


# Model Loading

In [24]:
# 전반적 스트럭처 결정하기 - 노드의 층수, 히든 유닛의 수
def build_model(hp):
    base_model = tf.keras.applications.MobileNetV2(
        input_shape=None,
        alpha=1.0,
        include_top=False,
        weights="imagenet",
        input_tensor=None,
        pooling=None
      )
    # 해당 층의 매개변수는 그대로 사용하기 위해서 freezing 을 한다.
    
    base_model.trainable = True
    inputs = keras.Input(shape=(320, 320, 3))
    x = base_model(inputs, training=True)
    x=keras.layers.GlobalAveragePooling2D()(x)

    # Tune the number of layers.
    for i in range(hp.Int("num_layers", 1, 5)):
        x=keras.layers.Dense(# Tune number of units separately.
                units=hp.Int(f"units_{i}", min_value=32, max_value=1024, step=32),
                activation=None)(x)
        tf.keras.layers.BatchNormalization()(x),
        tf.keras.layers.Activation(activation="elu")(x)
    if hp.Boolean("dropout"):
        x = keras.layers.Dropout(0.2)(x)
    outputs = keras.layers.Dense(3, activation='softmax')(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.00166) # MobilenetV2 의 optimal 한 learning rate 를 사용
    model.compile(optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(), # 교차엔트로피로 구성
              metrics=['accuracy'])
    
    return model

In [ ]:
# Create base model
# input_tensor 는 내가 쓰고 싶은 입력크기
# input_shape 는 모델이 가지는 입력크기를 말하는듯
''' [1] MobileNetV2 모델 
def build_model(hp):
    base_model = tf.keras.applications.MobileNetV2(
        input_shape=None,
        alpha=1.0,
        include_top=False,
        weights="imagenet",
        input_tensor=None,
        pooling=None
      )
    # 해당 층의 매개변수는 그대로 사용하기 위해서 freezing 을 한다.
    
    base_model.trainable = True
    inputs = keras.Input(shape=(320, 320, 3))
    x = base_model(inputs, training=True)
    x=keras.layers.GlobalAveragePooling2D()(x)

    # l1 과 l2 를 모두 적용을 시키므로, 큰 값은 더욱 작게하기위해 노력을 하고, 또한 0 엣지를 가지게 하기 유리한 L1 을 모두 적용한다.
    l1_rate = hp.Float("l1_rate", min_value=1e-5, max_value=1e-1, sampling="log")
    l2_rate = hp.Float("l2_rate", min_value=1e-5, max_value=1e-1, sampling="log")

    # Tune the number of layers.
    for i in range(hp.Int("num_layers", 1, 5)):
        # 정규화를 활성화 함수 이전에 배치하면 좋다고 수업시간에 한 학습을 바탕으로 모델링을 진행하였다.
        tf.keras.regularizers.l1_l2(l1=l1_rate, l2=l2_rate)
        x=keras.layers.Dense(# Tune number of units separately.
                units=hp.Int(f"units_{i}", min_value=32, max_value=512, step=32),
                activation=None, kernel_regularizer='l1_l2')(x)
        tf.keras.layers.BatchNormalization()(x),
        tf.keras.layers.Activation(activation=hp.Choice("activation", ["relu", "elu"]))(x)
    if hp.Boolean("dropout"):
        x = keras.layers.Dropout(0.2)(x)
    outputs = keras.layers.Dense(3, activation='softmax')(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    # Define the optimizer learning rate as a hyperparameter.
    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(), # 교차엔트로피로 구성
              metrics=['accuracy'])
    
    return model
'''

''' [2] Xception 모델
def build_model(hp):
    base_model = tf.keras.applications.Xception(
        include_top=False,
        weights="imagenet",
        input_tensor=None,
        input_shape=None)

    # 해당 층의 매개변수는 그대로 사용하기 위해서 freezing 을 한다.
    base_model.trainable = True
    inputs = keras.Input(shape=(320, 320, 3))
    x = base_model(inputs, training=True)
    x=keras.layers.GlobalAveragePooling2D()(x)

    # l1 과 l2 를 모두 적용을 시키므로, 큰 값은 더욱 작게하기위해 노력을 하고, 또한 0 엣지를 가지게 하기 유리한 L1 을 모두 적용한다.
    l1_rate = hp.Float("l1_rate", min_value=1e-5, max_value=1e-1, sampling="log")
    l2_rate = hp.Float("l2_rate", min_value=1e-5, max_value=1e-1, sampling="log")

    # Tune the number of layers.
    for i in range(hp.Int("num_layers", 1, 5)):
        # 정규화를 활성화 함수 이전에 배치하면 좋다고 수업시간에 한 학습을 바탕으로 모델링을 진행하였다.
        tf.keras.regularizers.l1_l2(l1=l1_rate, l2=l2_rate)
        x=keras.layers.Dense(# Tune number of units separately.
                units=hp.Int(f"units_{i}", min_value=32, max_value=512, step=32),
                activation=None, kernel_regularizer='l1_l2')(x)
        tf.keras.layers.BatchNormalization()(x),
        tf.keras.layers.Activation(activation=hp.Choice("activation", ["relu", "elu"]))(x)
    if hp.Boolean("dropout"):
        x = keras.layers.Dropout(0.2)(x)
    outputs = keras.layers.Dense(3, activation='softmax')(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    # Define the optimizer learning rate as a hyperparameter.
    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(), # 교차엔트로피로 구성
              metrics=['accuracy'])
    
    return model
'''

' [2] Xception 모델\ndef build_model(hp):\n    base_model = tf.keras.applications.Xception(\n        include_top=False,\n        weights="imagenet",\n        input_tensor=None,\n        input_shape=None)\n\n    # 해당 층의 매개변수는 그대로 사용하기 위해서 freezing 을 한다.\n    base_model.trainable = True\n    inputs = keras.Input(shape=(320, 320, 3))\n    x = base_model(inputs, training=True)\n    x=keras.layers.GlobalAveragePooling2D()(x)\n\n    # l1 과 l2 를 모두 적용을 시키므로, 큰 값은 더욱 작게하기위해 노력을 하고, 또한 0 엣지를 가지게 하기 유리한 L1 을 모두 적용한다.\n    l1_rate = hp.Float("l1_rate", min_value=1e-5, max_value=1e-1, sampling="log")\n    l2_rate = hp.Float("l2_rate", min_value=1e-5, max_value=1e-1, sampling="log")\n\n    # Tune the number of layers.\n    for i in range(hp.Int("num_layers", 1, 5)):\n        # 정규화를 활성화 함수 이전에 배치하면 좋다고 수업시간에 한 학습을 바탕으로 모델링을 진행하였다.\n        tf.keras.regularizers.l1_l2(l1=l1_rate, l2=l2_rate)\n        x=keras.layers.Dense(# Tune number of units separately.\n                units=hp.Int(f"units_{i}", min

In [25]:
# Keras_tuner 가 정상 작동하는지 확인
build_model(kt.HyperParameters())

In [ ]:
tuner = kt.BayesianOptimization(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=4,
    executions_per_trial=2,
    overwrite=True,
    directory="my_dir",
    project_name="helloworld",
)

In [ ]:
tuner.search_space_summary()

In [23]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

tuner.search(train_data_gen,validation_data=val_data_gen, epochs=7)


Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
num_layers        |2                 |?                 
units_0           |224               |?                 
dropout           |True              |?                 

Epoch 1/7


UnknownError: ignored

In [ ]:
tuner.results_summary()

Results summary
Results in my_dir/helloworld
Showing 10 best trials
Objective(name='val_accuracy', direction='max')


In [ ]:
# Create base model
# input_tensor 는 내가 쓰고 싶은 입력크기
# input_shape 는 모델이 가지는 입력크기를 말하는듯



strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
  base_model = tf.keras.applications.MobileNetV2(
    input_shape=None,
    alpha=1.0,
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    pooling=None
  )

# 해당 층의 매개변수는 그대로 사용하기 위해서 freezing 을 한다.
  base_model.trainable = True

  inputs = keras.Input(shape=(320, 320, 3))
  x = base_model(inputs, training=True)
  #x = keras.layers.Dropout(0.2)(x)
  x=keras.layers.GlobalAveragePooling2D()(x)
  x=keras.layers.Dense(64,activation='elu')(x)
  x = keras.layers.Dropout(0.4)(x)
  x=keras.layers.Dense(32,activation='elu')(x)
  x = keras.layers.Dropout(0.4)(x)
  outputs = keras.layers.Dense(3, activation='softmax')(x)

  model = keras.Model(inputs=inputs, outputs=outputs)
  model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])

In [ ]:
my_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=2),
    tf.keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5'),
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
]

In [ ]:
tf.debugging.set_log_device_placement(False)

In [ ]:
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=train_num//batch_size,     # 한 프로퍼게이션때 한 배치가 쭉 통과하므로 그렇군
    epochs= 3,
    validation_data = val_data_gen,
    validation_steps = val_num//batch_size,
    callbacks=[my_callbacks]
)